# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:

Use SFrames to do some feature engineering
Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
Look at coefficients and interpret their meanings
Evaluate multiple models via RSS

In [1]:
import graphlab

# Load in house sales data
Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame("/home/aunja/Downloads/kc_house_data.csv")

Finished parsing file /home/aunja/Downloads/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.238801 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/aunja/Downloads/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.181908 secs.

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results. In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).

In [4]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model
Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features: example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.102410     | 4146407.590959     | 258679.804259 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [6]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0692067  | 7873.33813675 |
| sqft_living |  None | 315.403441118  | 3.45570032294 |
|   bedrooms  |  None | -65080.2154416 | 2717.45685213 |
|  bathrooms  |  None | 6944.02087632  | 3923.11492813 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions
In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions.

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [7]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.504278


# Compute RSS
Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [8]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    residuals = outcome - predictions
    # Then square and add them up
    RSS = (residuals * residuals).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [9]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761534465e+14


# Create some new features
Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library


In [10]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:

* bedrooms_squared = bedrooms*bedrooms
* bed_bath_rooms = bedrooms*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long As an example here's the first one:



In [11]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [12]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4)   
  since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when both of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small   
  values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

In [13]:
train_data[['bedrooms','bathrooms','lat','long','bedrooms_squared','bed_bath_rooms','log_sqft_living','lat_plus_long']].head()

bedrooms,bathrooms,lat,long,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
3,1.0,47.5112,-122.257,9,3.0,7.07326971746,-74.7458
3,2.25,47.721,-122.319,9,6.75,7.85166117789,-74.598
2,1.0,47.7379,-122.233,4,2.0,6.64639051485,-74.4951
4,3.0,47.5208,-122.393,16,12.0,7.58069975222,-74.8722
3,2.0,47.6168,-122.045,9,6.0,7.4265490724,-74.4282
4,4.5,47.6561,-122.005,16,18.0,8.59785109443,-74.3489
3,2.25,47.3097,-122.327,9,6.75,7.4471683596,-75.0173
3,1.5,47.4095,-122.315,9,4.5,6.96602418711,-74.9055
3,1.0,47.5123,-122.337,9,3.0,7.48436864329,-74.8247
3,2.5,47.3684,-122.031,9,7.5,7.54433210805,-74.6626


# Q1-What is the mean value (arithmetic average) of the 'bedrooms_squared' feature on TEST data? (round to 2 decimal places)
# Q2-What is the mean value (arithmetic average) of the 'bed_bath_rooms' feature on TEST data? (round to 2 decimal places)
# Q3-What is the mean value (arithmetic average) of the 'log_sqft_living' feature on TEST data? (round to 2 decimal places)
# Q4-What is the mean value (arithmetic average) of the 'lat_plus_long' feature on TEST data? (round to 2 decimal places)

In [14]:
print test_data['bedrooms_squared'].mean()
print test_data['bed_bath_rooms'].mean()
print test_data['log_sqft_living'].mean()
print test_data['lat_plus_long'].mean()

12.4466777016
7.50390163159
7.55027467965
-74.653333554


# Learning Multiple Models
Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:

* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [15]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [16]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, 
                                                  validation_set = None)
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, 
                                                  validation_set = None)
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.109567     | 3193116.484463     | 228199.723038 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

# Q5-What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?
# Q6-What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?



In [18]:
# Examine/extract each model's coefficients:
model_1_weight_summary = model_1.get("coefficients")
model_2_weight_summary = model_2.get("coefficients")
model_3_weight_summary = model_3.get("coefficients")
print model_1_weight_summary 
print model_2_weight_summary
print model_3_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56141852.4692 | 1649928.35825 |
| sqft_living |  None | 310.263506724  | 3.18882062328 |
|   bedrooms  |  None | -59577.6052126 | 2487.27455846 |
|  bathrooms  |  None | 13812.4068763  | 3593.53440572 |
|     lat     |  None | 629863.119956  |  13120.68867  |
|     long    |  None | -214800.952332 | 13283.8203911 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]

+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -54412036.4015 | 1650346.09777 |
|  sqft_living   |  None |  304.44950524  | 3.20216518282 |
|    bedrooms    |  None | -116366.742819 | 4805.53398548 |
|   bathrooms    |  None | -77972.1056406 | 7565

* Ans5- Positive
* Ans6- Negative

# Q7-Which model (1, 2 or 3) has lowest RSS on TRAINING Data? Is this what you expected?

# Comparing multiple models
Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [19]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_model_1_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_model_2_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_model_3_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
print rss_model_1_train
print rss_model_2_train
print rss_model_3_train

9.7132387161e+14
9.61587552577e+14
9.05273774726e+14


Ans7- model_3

# Q8-Which model (1, 2 or 3) has lowest RSS on TESTING Data?

In [20]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_model_1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model_2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model_3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print rss_model_1_test
print rss_model_2_test
print rss_model_3_test


2.26566318597e+14
2.24366990581e+14
2.51824565049e+14


Ans8- model_2